In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

# Core

HierarchicalForecast contains pure Python implementations of hierarchical reconciliation methods as well as a `core.HierarchicalReconciliation` wrapper class that enables easy interaction with these methods through pandas DataFrames containing the hierarchical time series and the base predictions.

The `core.HierarchicalReconciliation` reconciliation class operates with the hierarchical time series pd.DataFrame `Y_df`, the base predictions pd.DataFrame `Y_hat_df`, the aggregation constraints matrix `S`. For more information on the creation of aggregation constraints matrix see the utils [aggregation method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br><br>

In [ ]:
#| export
import re
import gc
import time
import copy
from inspect import signature
from scipy.stats import norm
from scipy import sparse
from typing import Callable, Dict, List, Optional
import warnings

import numpy as np
import pandas as pd

In [ ]:
#| hide
from fastcore.test import test_close, test_eq, test_fail
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| exporti
def _build_fn_name(fn) -> str:
    fn_name = type(fn).__name__
    func_params = fn.__dict__

    # Take default parameter out of names
    args_to_remove = ['insample', 'num_threads']
    if not func_params.get('nonnegative', False):
        args_to_remove.append('nonnegative')

    if fn_name == 'MinTrace' and \
        func_params['method']=='mint_shrink':
        if func_params['mint_shr_ridge'] == 2e-8:
            args_to_remove += ['mint_shr_ridge']

    func_params = [f'{name}-{value}' for name, value in func_params.items() if name not in args_to_remove]
    if func_params:
        fn_name += '_' + '_'.join(func_params)
    return fn_name

In [ ]:
#| hide
# test fn name
from hierarchicalforecast.methods import BottomUp, MinTrace

In [ ]:
#| hide
test_eq(_build_fn_name(BottomUp()), 'BottomUp')
test_eq(
    _build_fn_name(MinTrace(method='ols')), 
    'MinTrace_method-ols'
)
test_eq(
    _build_fn_name(MinTrace(method='ols', nonnegative=True)), 
    'MinTrace_method-ols_nonnegative-True'
)
test_eq(
    _build_fn_name(MinTrace(method='mint_shr')), 
    'MinTrace_method-mint_shr'
)

# core.HierarchicalReconciliation

In [ ]:
#| exporti
def _reverse_engineer_sigmah(Y_hat_df, y_hat, model_name):
    """
    This function assumes that the model creates prediction intervals
    under a normality with the following the Equation:
    $\hat{y}_{t+h} + c \hat{sigma}_{h}$

    In the future, we might deprecate this function in favor of a 
    direct usage of an estimated $\hat{sigma}_{h}$
    """

    drop_cols = ['ds']
    if 'y' in Y_hat_df.columns:
        drop_cols.append('y')
    if model_name+'-median' in Y_hat_df.columns:
        drop_cols.append(model_name+'-median')
    model_names = Y_hat_df.drop(columns=drop_cols, axis=1).columns.to_list()
    pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name)]
    pi_model_name = [pi_name for pi_name in pi_model_names if model_name in pi_name]
    pi = len(pi_model_name) > 0

    n_series = len(Y_hat_df.index.unique())

    if not pi:
        raise Exception(f'Please include `{model_name}` prediction intervals in `Y_hat_df`')

    pi_col = pi_model_name[0]
    sign = -1 if 'lo' in pi_col else 1
    level_col = re.findall('[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', pi_col)
    level_col = float(level_col[-1])
    z = norm.ppf(0.5 + level_col / 200)
    sigmah = Y_hat_df[pi_col].values.reshape(n_series,-1)
    sigmah = sign * (sigmah - y_hat) / z

    return sigmah

In [ ]:
#| export
class HierarchicalReconciliation:
    """Hierarchical Reconciliation Class.

    The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
    HierarchicaForecast methods for a collection of time series and base predictions stored in 
    pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
    y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
    example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

    **Parameters:**<br>
    `reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

    **References:**<br>
    [Rob J. Hyndman and George Athanasopoulos (2018). \"Forecasting principles and practice, Hierarchical and Grouped Series\".](https://otexts.com/fpp3/hierarchical.html)
    """
    def __init__(self,
                 reconcilers: List[Callable]):
        self.reconcilers = reconcilers
        self.orig_reconcilers = copy.deepcopy(reconcilers) # TODO: elegant solution
        self.insample = any([method.insample for method in reconcilers])
    
    def _prepare_fit(self,
                     Y_hat_df: pd.DataFrame,
                     S_df: pd.DataFrame,
                     Y_df: Optional[pd.DataFrame],
                     tags: Dict[str, np.ndarray],
                     level: Optional[List[int]] = None,
                     intervals_method: str = 'normality',
                     sort_df: bool = True):
        """
        Performs preliminary wrangling and protections
        """
        #-------------------------------- Match Y_hat/Y/S index order --------------------------------#
        if sort_df:
            Y_hat_df = Y_hat_df.reset_index()
            Y_hat_df.unique_id = Y_hat_df.unique_id.astype('category')
            Y_hat_df.unique_id = Y_hat_df.unique_id.cat.set_categories(S_df.index)
            Y_hat_df = Y_hat_df.sort_values(by=['unique_id', 'ds'])
            Y_hat_df = Y_hat_df.set_index('unique_id')

            if Y_df is not None:
                Y_df = Y_df.reset_index()
                Y_df.unique_id = Y_df.unique_id.astype('category')
                Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
                Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
                Y_df = Y_df.set_index('unique_id')

            S_df.index = pd.CategoricalIndex(S_df.index, categories=S_df.index)

        #----------------------------------- Check Input's Validity ----------------------------------#
        # Check input's validity
        if intervals_method not in ['normality', 'bootstrap', 'permbu']:
            raise ValueError(f'Unkwon interval method: {intervals_method}')

        if self.insample or (intervals_method in ['bootstrap', 'permbu']):
            if Y_df is None:
                raise Exception('you need to pass `Y_df`')
        
        # Protect level list
        if (level is not None):
            level_outside_domain = np.any((np.array(level) < 0)|(np.array(level) >= 100 ))
            if level_outside_domain and (intervals_method in ['normality', 'permbu']):
                raise Exception('Level outside domain, send `level` list in [0,100)')

        # Declare output names
        drop_cols = ['ds', 'y'] if 'y' in Y_hat_df.columns else ['ds']
        model_names = Y_hat_df.drop(columns=drop_cols, axis=1).columns.to_list()

        # Ensure numeric columns
        if not len(Y_hat_df[model_names].select_dtypes(include='number').columns) == len(Y_hat_df[model_names].columns):
            raise Exception('`Y_hat_df`s columns contain non numeric types')
            
        #Ensure no null values
        if Y_hat_df[model_names].isnull().values.any():
            raise Exception('`Y_hat_df` contains null values')
        
        pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name or '-median' in name)]
        model_names = [name for name in model_names if name not in pi_model_names]
        
        # TODO: Complete y_hat_insample protection
        if intervals_method in ['bootstrap', 'permbu']:
            if not (set(model_names) <= set(Y_df.columns)):
                raise Exception('Check `Y_hat_df`s models are included in `Y_df` columns')

        uids = Y_hat_df.index.unique()

        # Check Y_hat_df\S_df series difference
        S_diff  = len(S_df.index.difference(uids))
        Y_hat_diff = len(Y_hat_df.index.difference(S_df.index.unique()))
        if S_diff > 0 or Y_hat_diff > 0:
            raise Exception(f'Check `S_df`, `Y_hat_df` series difference, S\Y_hat={S_diff}, Y_hat\S={Y_hat_diff}')

        if Y_df is not None:
            # Check Y_hat_df\Y_df series difference
            Y_diff  = len(Y_df.index.difference(uids))
            Y_hat_diff = len(Y_hat_df.index.difference(Y_df.index.unique()))
            if Y_diff > 0 or Y_hat_diff > 0:
                raise Exception(f'Check `Y_hat_df`, `Y_df` series difference, Y_hat\Y={Y_hat_diff}, Y\Y_hat={Y_diff}')

        # Same Y_hat_df/S_df/Y_df's unique_id order to prevent errors
        S_df = S_df.loc[uids]

        return Y_hat_df, S_df, Y_df, model_names

    def reconcile(self, 
                  Y_hat_df: pd.DataFrame,
                  S: pd.DataFrame,
                  tags: Dict[str, np.ndarray],
                  Y_df: Optional[pd.DataFrame] = None,
                  level: Optional[List[int]] = None,
                  intervals_method: str = 'normality',
                  num_samples: int = -1,
                  seed: int = 0,
                  sort_df: bool = True,
                  is_balanced: bool = False,
        ):
        """Hierarchical Reconciliation Method.

        The `reconcile` method is analogous to SKLearn `fit_predict` method, it 
        applies different reconciliation techniques instantiated in the `reconcilers` list.

        Most reconciliation methods can be described by the following convenient 
        linear algebra notation:

        $$\\tilde{\mathbf{y}}_{[a,b],\\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\\tau}$$

        where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
        the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
        reconciliation methods. The reconciled predictions are $\\tilde{\mathbf{y}}_{[a,b],\\tau}$, and the 
        base predictions $\hat{\mathbf{y}}_{[a,b],\\tau}$.

        **Parameters:**<br>
        `Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
        `Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.<br>
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: positive float list [0,100), confidence levels for prediction intervals.<br>
        `intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
        `num_samples`: int=-1, if positive return that many probabilistic coherent samples.
        `seed`: int=0, random seed for numpy generator's replicability.<br>
        `sort_df` : bool (default=True), if True, sort `df` by [`unique_id`,`ds`].<br>
        `is_balanced`: bool=False, wether `Y_df` is balanced, set it to True to speed things up if `Y_df` is balanced.<br>

        **Returns:**<br>
        `Y_tilde_df`: pd.DataFrame, with reconciled predictions.
        """
        # Check input's validity and sort dataframes
        Y_hat_df, S_df, Y_df, self.model_names = \
                    self._prepare_fit(Y_hat_df=Y_hat_df,
                                      S_df=S,
                                      Y_df=Y_df,
                                      tags=tags,
                                      level=level,
                                      intervals_method=intervals_method,
                                      sort_df=sort_df)

        # Initialize reconciler arguments
        reconciler_args = dict(
            idx_bottom=S_df.index.get_indexer(S.columns),
            tags={key: S_df.index.get_indexer(val) for key, val in tags.items()}
        )

        any_sparse = any([method.is_sparse_method for method in self.reconcilers])
        if any_sparse:
            try:
                S_for_sparse = sparse.csr_matrix(S_df.sparse.to_coo())
            except AttributeError:
                warnings.warn('Using dense S matrix for sparse reconciliation method.')
                S_for_sparse = S_df.values.astype(np.float32)

        if Y_df is not None:
            if is_balanced:
                y_insample = Y_df['y'].values.reshape(len(S_df), -1).astype(np.float32)
            else:
                y_insample = Y_df.pivot(columns='ds', values='y').loc[S_df.index].values.astype(np.float32)
            reconciler_args['y_insample'] = y_insample

        Y_tilde_df= Y_hat_df.copy()
        start = time.time()
        self.execution_times = {}
        self.level_names = {}
        self.sample_names = {}
        for reconcile_fn, name_copy in zip(self.reconcilers, self.orig_reconcilers):
            reconcile_fn_name = _build_fn_name(name_copy)

            if reconcile_fn.is_sparse_method:
                reconciler_args["S"] = S_for_sparse
            else:
                reconciler_args["S"] = S_df.values.astype(np.float32)

            has_fitted = 'y_hat_insample' in signature(reconcile_fn).parameters
            has_level = 'level' in signature(reconcile_fn).parameters

            for model_name in self.model_names:
                recmodel_name = f'{model_name}/{reconcile_fn_name}'
                y_hat = Y_hat_df[model_name].values.reshape(len(S_df), -1).astype(np.float32)
                reconciler_args['y_hat'] = y_hat

                if (self.insample and has_fitted) or intervals_method in ['bootstrap', 'permbu']:
                    if is_balanced:
                        y_hat_insample = Y_df[model_name].values.reshape(len(S_df), -1).astype(np.float32)
                    else:
                        y_hat_insample = Y_df.pivot(columns='ds', values=model_name).loc[S_df.index].values.astype(np.float32)
                    reconciler_args['y_hat_insample'] = y_hat_insample

                if has_level and (level is not None):
                    if intervals_method in ['normality', 'permbu']:
                        sigmah = _reverse_engineer_sigmah(Y_hat_df=Y_hat_df,
                                    y_hat=y_hat, model_name=model_name)
                        reconciler_args['sigmah'] = sigmah

                    reconciler_args['intervals_method'] = intervals_method
                    reconciler_args['num_samples'] = 200 # TODO: solve duplicated num_samples
                    reconciler_args['seed'] = seed

                # Mean and Probabilistic reconciliation
                kwargs = [key for key in signature(reconcile_fn).parameters if key in reconciler_args.keys()]
                kwargs = {key: reconciler_args[key] for key in kwargs}
                
                if (level is not None) and (num_samples > 0):
                    # Store reconciler's memory to generate samples
                    reconciler = reconcile_fn.fit(**kwargs)
                    fcsts_model = reconciler.predict(S=reconciler_args['S'], 
                                                     y_hat=reconciler_args['y_hat'], level=level)
                else:
                    # Memory efficient reconciler's fit_predict
                    fcsts_model = reconcile_fn(**kwargs, level=level)

                # Parse final outputs
                Y_tilde_df[recmodel_name] = fcsts_model['mean'].flatten()
                if intervals_method in ['bootstrap', 'normality', 'permbu'] and level is not None:
                    level.sort()
                    lo_names = [f'{recmodel_name}-lo-{lv}' for lv in reversed(level)]
                    hi_names = [f'{recmodel_name}-hi-{lv}' for lv in level]
                    self.level_names[recmodel_name] = lo_names + hi_names
                    sorted_quantiles = np.reshape(fcsts_model['quantiles'], (len(Y_tilde_df),-1))
                    intervals_df = pd.DataFrame(sorted_quantiles, index=Y_tilde_df.index,
                                                columns=self.level_names[recmodel_name])
                    Y_tilde_df= pd.concat([Y_tilde_df, intervals_df], axis=1)

                    if num_samples > 0:
                        samples = reconciler.sample(num_samples=num_samples)
                        self.sample_names[recmodel_name] = [f'{recmodel_name}-sample-{i}' for i in range(num_samples)]
                        samples = np.reshape(samples, (len(Y_tilde_df),-1))
                        samples_df = pd.DataFrame(samples, index=Y_tilde_df.index,
                                                  columns=self.sample_names[recmodel_name])
                        Y_tilde_df= pd.concat([Y_tilde_df, samples_df], axis=1)

                    del sorted_quantiles
                    del intervals_df
                if self.insample and has_fitted:
                    del y_hat_insample
                gc.collect()

                end = time.time()
                self.execution_times[f'{model_name}/{reconcile_fn_name}'] = (end - start)

        return Y_tilde_df

    def bootstrap_reconcile(self,
                            Y_hat_df: pd.DataFrame,
                            S_df: pd.DataFrame,
                            tags: Dict[str, np.ndarray],
                            Y_df: Optional[pd.DataFrame] = None,
                            level: Optional[List[int]] = None,
                            intervals_method: str = 'normality',
                            num_samples: int = -1,
                            num_seeds: int = 1,
                            sort_df: bool = True):
        """Bootstraped Hierarchical Reconciliation Method.

        Applies N times, based on different random seeds, the `reconcile` method 
        for the different reconciliation techniques instantiated in the `reconcilers` list. 

        **Parameters:**<br>
        `Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
        `Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.<br>
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: positive float list [0,100), confidence levels for prediction intervals.<br>
        `intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
        `num_samples`: int=-1, if positive return that many probabilistic coherent samples.
        `num_seeds`: int=1, random seed for numpy generator's replicability.<br>
        `sort_df` : bool (default=True), if True, sort `df` by [`unique_id`,`ds`].<br>

        **Returns:**<br>
        `Y_bootstrap_df`: pd.DataFrame, with bootstraped reconciled predictions.
        """

        # Check input's validity and sort dataframes
        Y_hat_df, S_df, Y_df, self.model_names = \
                    self._prepare_fit(Y_hat_df=Y_hat_df,
                                      S_df=S_df,
                                      Y_df=Y_df,
                                      tags=tags,
                                      intervals_method=intervals_method,
                                      sort_df=sort_df)

        # Bootstrap reconciled predictions
        Y_tilde_list = []
        for seed in range(num_seeds):
            Y_tilde_df = self.reconcile(Y_hat_df=Y_hat_df,
                                        S=S_df,
                                        tags=tags,
                                        Y_df=Y_df,
                                        level=level,
                                        intervals_method=intervals_method,
                                        num_samples=num_samples,
                                        seed=seed,
                                        sort_df=False)
            Y_tilde_df['seed'] = seed
            # TODO: fix broken recmodel_names
            if seed==0:
                first_columns = Y_tilde_df.columns
            Y_tilde_df.columns = first_columns
            Y_tilde_list.append(Y_tilde_df)

        Y_bootstrap_df = pd.concat(Y_tilde_list, axis=0)
        del Y_tilde_list
        gc.collect()

        return Y_bootstrap_df

In [ ]:
show_doc(HierarchicalReconciliation, 
         name='init', title_level=3)

---

### init

>      init (reconcilers:List[Callable])

Hierarchical Reconciliation Class.

The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
HierarchicaForecast methods for a collection of time series and base predictions stored in 
pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

**Parameters:**<br>
`reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

**References:**<br>
[Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Hierarchical and Grouped Series".](https://otexts.com/fpp3/hierarchical.html)

In [ ]:
show_doc(HierarchicalReconciliation)

---

### HierarchicalReconciliation

>      HierarchicalReconciliation (reconcilers:List[Callable])

Hierarchical Reconciliation Class.

The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
HierarchicaForecast methods for a collection of time series and base predictions stored in 
pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

**Parameters:**<br>
`reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

**References:**<br>
[Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Hierarchical and Grouped Series".](https://otexts.com/fpp3/hierarchical.html)

In [ ]:
show_doc(HierarchicalReconciliation.reconcile, name='reconcile', title_level=3)

---

### reconcile

>      reconcile (Y_hat_df:pandas.core.frame.DataFrame,
>                 S:pandas.core.frame.DataFrame, tags:Dict[str,numpy.ndarray],
>                 Y_df:Optional[pandas.core.frame.DataFrame]=None,
>                 level:Optional[List[int]]=None,
>                 intervals_method:str='normality', num_samples:int=-1,
>                 seed:int=0, sort_df:bool=True, is_balanced:bool=False)

Hierarchical Reconciliation Method.

The `reconcile` method is analogous to SKLearn `fit_predict` method, it 
applies different reconciliation techniques instantiated in the `reconcilers` list.

Most reconciliation methods can be described by the following convenient 
linear algebra notation:

$$\tilde{\mathbf{y}}_{[a,b],\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\tau}$$

where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
reconciliation methods. The reconciled predictions are $\tilde{\mathbf{y}}_{[a,b],\tau}$, and the 
base predictions $\hat{\mathbf{y}}_{[a,b],\tau}$.

**Parameters:**<br>
`Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
`Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.<br>
If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
`S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
`tags`: Each key is a level and its value contains tags associated to that level.<br>
`level`: positive float list [0,100), confidence levels for prediction intervals.<br>
`intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
`num_samples`: int=-1, if positive return that many probabilistic coherent samples.
`seed`: int=0, random seed for numpy generator's replicability.<br>
`sort_df` : bool (default=True), if True, sort `df` by [`unique_id`,`ds`].<br>
`is_balanced`: bool=False, wether `Y_df` is balanced, set it to True to speed things up if `Y_df` is balanced.<br>

**Returns:**<br>
`Y_tilde_df`: pd.DataFrame, with reconciled predictions.

In [ ]:
show_doc(HierarchicalReconciliation.bootstrap_reconcile, name='bootstrap_reconcile', title_level=3)

---

### bootstrap_reconcile

>      bootstrap_reconcile (Y_hat_df:pandas.core.frame.DataFrame,
>                           S_df:pandas.core.frame.DataFrame,
>                           tags:Dict[str,numpy.ndarray],
>                           Y_df:Optional[pandas.core.frame.DataFrame]=None,
>                           level:Optional[List[int]]=None,
>                           intervals_method:str='normality',
>                           num_samples:int=-1, num_seeds:int=1,
>                           sort_df:bool=True)

Bootstraped Hierarchical Reconciliation Method.

Applies N times, based on different random seeds, the `reconcile` method 
for the different reconciliation techniques instantiated in the `reconcilers` list. 

**Parameters:**<br>
`Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
`Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.<br>
If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
`S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
`tags`: Each key is a level and its value contains tags associated to that level.<br>
`level`: positive float list [0,100), confidence levels for prediction intervals.<br>
`intervals_method`: str, method used to calculate prediction intervals, one of `normality`, `bootstrap`, `permbu`.<br>
`num_samples`: int=-1, if positive return that many probabilistic coherent samples.
`num_seeds`: int=1, random seed for numpy generator's replicability.<br>
`sort_df` : bool (default=True), if True, sort `df` by [`unique_id`,`ds`].<br>

**Returns:**<br>
`Y_bootstrap_df`: pd.DataFrame, with bootstraped reconciled predictions.

In [ ]:
#| hide
from hierarchicalforecast.methods import (
    BottomUp, TopDown, MiddleOut, MinTrace, ERM,
)
from hierarchicalforecast.utils import aggregate

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# non strictly hierarchical structure
hierS_grouped_df = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped_df, tags_grouped = aggregate(df, hierS_grouped_df)
hier_strict_df, S_strict, tags_strict = aggregate(df, hiers_strictly)

# check categorical input produces same output
df2 = df.copy()
for col in ['Country', 'State', 'Purpose', 'Region']:
    df2[col] = df2[col].astype('category')

for spec in [hierS_grouped_df, hiers_strictly]:
    Y_orig, S_orig, tags_orig = aggregate(df, spec)
    Y_cat, S_cat, tags_cat = aggregate(df2, spec)
    pd.testing.assert_frame_equal(Y_cat, Y_orig)
    pd.testing.assert_frame_equal(S_cat, S_orig)
    assert all(np.array_equal(tags_orig[k], tags_cat[k]) for k in tags_orig.keys())

In [ ]:
#| hide
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_hat_df = hier_grouped_df.groupby('unique_id').tail(12)
ds_h = hier_grouped_hat_df['ds'].unique()
hier_grouped_df = hier_grouped_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_grouped_df['y_model'] += np.random.uniform(-1, 1, len(hier_grouped_df))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df, Y_df=hier_grouped_df, 
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# test incorrect Y_hat_df datatypes
hier_grouped_hat_df_nan = hier_grouped_hat_df.copy()
hier_grouped_hat_df_nan.loc['Australia', 'y_model'] = float('nan')
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_nan, S_grouped_df, tags_grouped, hier_grouped_df),
)

hier_grouped_hat_df_none = hier_grouped_hat_df.copy()
hier_grouped_hat_df_none.loc['Australia', 'y_model'] = None
test_fail(
    hrec.reconcile,
    contains='null values',
    args=(hier_grouped_hat_df_none, S_grouped_df, tags_grouped, hier_grouped_df),
)

hier_grouped_hat_df_str = hier_grouped_hat_df.copy()
hier_grouped_hat_df_str['y_model'] = hier_grouped_hat_df_str['y_model'].astype(str)
test_fail(
    hrec.reconcile,
    contains='numeric types',
    args=(hier_grouped_hat_df_str, S_grouped_df, tags_grouped, hier_grouped_df),
)

In [ ]:
#| hide
# test expected error
# different series S and Y_hat_df
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df.drop('Australia'), S_grouped_df, tags_grouped, hier_grouped_df),
    
)
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df, S_grouped_df.drop('Australia'), tags_grouped, hier_grouped_df),
    
)
test_fail(
    hrec.reconcile,
    contains='series difference',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df.drop('Australia')),
    
)

In [ ]:
# | hide
# test expected error
# different columns Y_df and Y_hat_df
hrec = HierarchicalReconciliation(
            reconcilers=[ERM(method='reg_bu', lambda_reg=100)])
test_fail(
    hrec.reconcile,
    contains='Please include ',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, 
          hier_grouped_df, [80], 'permbu'), # permbu needs y_hat_insample
)

In [ ]:
#| hide
# test reconcile method without insample
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df,
                            S=S_grouped_df, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# top down should break
# with non strictly hierarchical structures
hrec = HierarchicalReconciliation([TopDown(method='average_proportions')])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  hier_grouped_df,)
)

In [ ]:
#| hide
# methods should work with
# srtictly hierarchical structures
#| hide
hier_strict_df['y_model'] = hier_strict_df['y']
# we should be able to recover y using the methods
hier_strict_df_h = hier_strict_df.groupby('unique_id').tail(12)
ds_h = hier_strict_df_h['ds'].unique()
hier_strict_df = hier_strict_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_strict_df['y_model'] += np.random.uniform(-1, 1, len(hier_strict_df))

middle_out_level = 'Country/State'
# hierarchical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # top down doesnt recover the original y
    # but it should recover the total level
    TopDown(method='forecast_proportions'),
    TopDown(method='average_proportions'),
    TopDown(method='proportion_averages'),
    # middle out doesnt recover the original y
    # but it should recover the total level
    MiddleOut(middle_level=middle_out_level, top_down_method='forecast_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='average_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='proportion_averages'),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    elif 'nonnegative' in model:
        eps = 1e-1
    else:
        eps = 1e-1
    if 'TopDown' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict['Country']
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    elif 'MiddleOut' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict[middle_out_level]
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    else:
        test_close(reconciled['y'], reconciled[model], eps)

In [ ]:
#| hide
# test is_balanced behaviour
reconciled_balanced = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict,
    is_balanced=True,
)
test_close(reconciled.drop(columns='ds').values, reconciled_balanced.drop(columns='ds').values, eps=1e-10)

In [ ]:
#| hide
from statsforecast import StatsForecast
from statsforecast.utils import generate_series
from statsforecast.models import RandomWalkWithDrift

In [ ]:
#| hide
# test unbalanced dataset
max_tenure = 24
dates = pd.date_range(start='2019-01-31', freq='M', periods=max_tenure)
cohort_tenure = [24, 23, 22, 21]

ts_list = []

# Create ts for each cohort
for i in range(len(cohort_tenure)):
    ts_list.append(
        generate_series(n_series=1, freq='M', min_length=cohort_tenure[i], max_length=cohort_tenure[i]).reset_index() \
            .assign(ult=i) \
            .assign(ds=dates[-cohort_tenure[i]:]) \
            .drop(columns=['unique_id'])
    )
df = pd.concat(ts_list, ignore_index=True)

# Create categories
df.loc[df['ult'] < 2, 'pen'] = 'a'
df.loc[df['ult'] >= 2, 'pen'] = 'b'
# Note that unique id requires strings
df['ult'] = df['ult'].astype(str)

hier_levels = [
    ['pen'],
    ['pen', 'ult'],
]
hier_df, S_df, tags = aggregate(df=df, spec=hier_levels)

train_df = hier_df.query("ds <= @pd.to_datetime('2019-12-31')")
test_df = hier_df.query("ds > @pd.to_datetime('2019-12-31')")

fcst = StatsForecast(
    models=[
        RandomWalkWithDrift(),
    ],
    freq='M',
    n_jobs=1,
)

hrec = HierarchicalReconciliation(
    reconcilers=[
        BottomUp(),
        MinTrace(method='mint_shrink'),
    ]
)

fcst_df = fcst.forecast(df=train_df, h=12, fitted=True)
fitted_df = fcst.forecast_fitted_values()

fcst_df = hrec.reconcile(
    Y_hat_df=fcst_df,
    Y_df=fitted_df,
    S=S_df,
    tags=tags,
)

In [ ]:
#| hide
# MinTrace should break
# with extremely overfitted model, y_model==y

zero_df = hier_grouped_df.copy()
zero_df['y'] = 0
zero_df['y_model'] = 0
hrec = HierarchicalReconciliation([MinTrace(method='mint_shrink')])
test_fail(
    hrec.reconcile,
    contains='Insample residuals',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped,  zero_df)
)

In [ ]:
#| hide
#test methods that dont use residuals
#even if their signature includes
#that argument
hrec = HierarchicalReconciliation([MinTrace(method='ols')])
reconciled = hrec.reconcile(
    Y_hat_df=hier_grouped_hat_df, 
    Y_df=hier_grouped_df.drop(columns=['y_model']), 
    S=S_grouped_df, 
    tags=tags_grouped
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    test_close(reconciled['y'], reconciled[model], eps=1e-1)

In [ ]:
#| hide
reconciled.loc[tags_grouped['Country/State']]

,ds,y,y_model,y_model/MinTrace_method-ols
unique_id,,,,
Australia/ACT,2015 Q1,566.135463,566.135463,566.135431
Australia/ACT,2015 Q2,516.870343,516.870343,516.870360
Australia/ACT,2015 Q3,688.203188,688.203188,688.203222
Australia/ACT,2015 Q4,597.245569,597.245569,597.245600
Australia/ACT,2016 Q1,625.141634,625.141634,625.141589
...,...,...,...,...
Australia/Western Australia,2016 Q4,2656.330701,2656.330701,2656.330828
Australia/Western Australia,2017 Q1,2570.911689,2570.911689,2570.911638
Australia/Western Australia,2017 Q2,2438.487939,2438.487939,2438.488056


In [ ]:
#| hide
# test methods with bootstrap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df, 
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='bootstrap')
total = reconciled.loc[tags_grouped['Country/State/Region/Purpose']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with  normality prediction intervals
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_hat_df,
                            Y_df=hier_grouped_df, S=S_grouped_df, tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='normality')
total = reconciled.loc[tags_grouped['Country/State/Region/Purpose']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with PERMBU prediction intervals

# test expect error with grouped structure
# (non strictly hierarchical)
hier_grouped_hat_df['y_model-lo-80'] = hier_grouped_hat_df['y_model'] - 1.96
hier_grouped_hat_df['y_model-hi-80'] = hier_grouped_hat_df['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [80, 90], 'permbu',)
)

# test PERMBU
hier_strict_df_h['y_model-lo-80'] = hier_strict_df_h['y_model'] - 1.96
hier_strict_df_h['y_model-hi-80'] = hier_strict_df_h['y_model'] + 1.96
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(Y_hat_df=hier_strict_df_h,
                            Y_df=hier_strict_df, S=S_strict, 
                            tags=tags_grouped,
                            level=[80, 90], 
                            intervals_method='permbu')
total = reconciled.loc[tags_grouped['Country/State/Region']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)
assert 'y_model/BottomUp-lo-80' in reconciled.columns

In [ ]:
#| hide
# test methods with Bootraped Bootstap prediction intervals
hrec = HierarchicalReconciliation([BottomUp()])
bootstrap_df = hrec.bootstrap_reconcile(Y_hat_df=hier_grouped_hat_df,
                                        Y_df=hier_grouped_df, S_df=S_grouped_df, tags=tags_grouped,
                                        level=[80, 90],
                                        intervals_method='bootstrap',
                                        num_seeds=2)
assert 'y_model/BottomUp-lo-80' in bootstrap_df.columns
assert 'seed' in bootstrap_df.columns
assert len(bootstrap_df.seed.unique())==2
bootstrap_df

,ds,y,y_model,y_model-lo-80,y_model-hi-80,y_model/BottomUp,y_model/BottomUp-lo-90,y_model/BottomUp-lo-80,y_model/BottomUp-hi-80,y_model/BottomUp-hi-90,seed
unique_id,,,,,,,,,,,
Australia,2015 Q1,25023.736745,25023.736745,25021.776745,25025.696745,25023.740234,25008.027344,25010.248047,25037.162109,25039.646777,0
Australia,2015 Q2,23798.914367,23798.914367,23796.954367,23800.874367,23798.919922,23781.757812,23784.914062,23811.863281,23815.722656,0
Australia,2015 Q3,23485.745655,23485.745655,23483.785655,23487.705655,23485.744141,23468.589844,23471.744141,23498.741211,23502.607031,0
Australia,2015 Q4,25140.161221,25140.161221,25138.201221,25142.121221,25140.162109,25123.005859,25126.673828,25153.696094,25156.970703,0
Australia,2016 Q1,26660.637689,26660.637689,26658.677690,26662.597689,26660.628906,26645.546875,26648.501367,26674.172656,26677.497559,0
...,...,...,...,...,...,...,...,...,...,...,...
Australia/Western Australia/Experience Perth/Visiting,2016 Q4,439.699451,439.699451,437.739451,441.659450,439.699463,438.771545,438.851196,440.507257,440.618561,1
Australia/Western Australia/Experience Perth/Visiting,2017 Q1,356.867038,356.867038,354.907038,358.827038,356.867035,356.015839,356.050018,357.678070,357.721121,1
Australia/Western Australia/Experience Perth/Visiting,2017 Q2,302.296119,302.296119,300.336119,304.256119,302.296112,301.388199,301.465564,303.170898,303.240173,1


In [ ]:
#| hide
# test level protection for PERMBU and Normality probabilistic methods
hrec = HierarchicalReconciliation([BottomUp()])
test_fail(
    hrec.reconcile,
    contains='Level outside domain',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [-1, 80, 90], 'permbu',)
)
test_fail(
    hrec.reconcile,
    contains='Level outside domain',
    args=(hier_grouped_hat_df, S_grouped_df, tags_grouped, hier_grouped_df, [80, 90, 101], 'normality',)
)

# Example

In [ ]:
#| eval: false
import numpy as np
import pandas as pd

from statsforecast.core import StatsForecast
from statsforecast.models import ETS, Naive

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace

# Load TourismSmall dataset
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# Create hierarchical seires based on geographic levels and purpose
# And Convert quarterly ds string to pd.datetime format
hierarchy_levels = [['Country'],
                    ['Country', 'State'], 
                    ['Country', 'Purpose'], 
                    ['Country', 'State', 'Region'], 
                    ['Country', 'State', 'Purpose'], 
                    ['Country', 'State', 'Region', 'Purpose']]

Y_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)
qs = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()
Y_df = Y_df.reset_index()

# Split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(4)
Y_train_df = Y_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(models=[Naive()], freq='Q', n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=4, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

# Reconcile the base predictions
Y_train_df = Y_train_df.reset_index().set_index('unique_id')
Y_hat_df = Y_hat_df.reset_index().set_index('unique_id')
reconcilers = [BottomUp(),
               MinTrace(method='mint_shrink')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, 
                          Y_df=Y_fitted_df,
                          S=S_df, tags=tags)
Y_rec_df.groupby('unique_id', observed=True).head(2)

,ds,Naive,Naive/BottomUp,Naive/MinTrace_method-mint_shrink
unique_id,,,,
Australia,2016-12-31,26347.601562,26347.583984,26347.600970
Australia,2017-03-31,26347.601562,26347.583984,26347.600970
Australia/ACT,2016-12-31,667.214111,667.214111,667.214126
Australia/ACT,2017-03-31,667.214111,667.214111,667.214126
Australia/New South Wales,2016-12-31,7995.501953,7995.502930,7995.502137
...,...,...,...,...
Australia/Western Australia/Experience Perth/Holiday,2017-03-31,310.973419,310.973419,310.973408
Australia/Western Australia/Experience Perth/Other,2016-12-31,89.863976,89.863976,89.863975
Australia/Western Australia/Experience Perth/Other,2017-03-31,89.863976,89.863976,89.863975
